In [34]:
%%time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup # Превращалка html в текст.

%matplotlib inline

df_sample = pd.read_csv('sampleSubmission.csv', sep = '\t')
df_train = pd.read_csv('train.csv', sep = '\t')
df_test = pd.read_csv('test.csv', sep = '\t')
print('df_train ',df_train.shape)
print('df_test ',df_test.shape)

df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'test'
df = df_test.append(df_train).reset_index(drop=True)
print('df ',df.shape)

df_dict = df[['description']].to_dict()['description']
df_name = df[['name']].to_dict()['name']
print('df_dict ',len(df_dict))

del df_dict
del df
del df_sample
del df_train
del df_test

df_train  (200000, 4)
df_test  (170179, 3)


/Users/dinar/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


df  (370179, 5)
df_dict  370179
CPU times: user 10.3 s, sys: 2.85 s, total: 13.2 s
Wall time: 13.7 s


In [35]:
print('df_name ',len(df_name))

df_name  370179


# Посчитаем лишние слова

In [79]:
%%time

words = {}
for line in list(df_dict_re.keys()):
    for word in df_dict_re[line].split():
        if words.get(word):
            words[word] += 1
        else:
            words[word] = 1
print(len(words))

df_words = pd.DataFrame(list(words.items())).sort_values(by=1, ascending=False).reset_index(drop=True)
df_words['len_0'] = df_words[0].map(lambda x: len(x) )
df_block_words = df_words[df_words['len_0'] <4] # лишние слова - когда длина слов меньше или равно 3-х

CPU times: user 29.3 s, sys: 1.83 s, total: 31.1 s
Wall time: 32.1 s


# Метод 1: подсчет слов вручную в словарь и перевод в DataFrame

In [ ]:
%%time

# pstrong = re.compile('<p><strong>(.+?)</strong></p>', re.S)
# p = re.compile('<p>.+?</p>', re.S)
# ulli = re.compile("<ul>\ <li>(.+?)</li>\ </ul>", re.S)
# ''.join(ulli.split(''.join(p.split(''.join(pstrong.split(df_dict[line]))))))

df_dict_re = {}
for line in df_dict.keys():
    df_dict_re[line] = ' '.join(re.findall("[А-ЯЁа-яё]+[\:|\,|\.]*", df_dict[line]))
print('df_dict_re ',len(df_dict_re))

In [51]:
%%time

word_to_number_for_each_texts = {}
for text in df_dict_re.values():
    word_to_number_for_each_texts[text] = {}
    for word in text.split():
        if word_to_number_for_each_texts[text].get(word):
            word_to_number_for_each_texts[text][word] += 1
        else:
            word_to_number_for_each_texts[text][word] = 1
len(word_to_number_for_each_texts)

%%time

CPU times: user 25.6 s, sys: 2.17 s, total: 27.7 s
Wall time: 28.2 s


In [116]:
%%time

# df_dict_re_matrix = pd.DataFrame(list(word_to_number_for_each_texts.values())).fillna(0)
# df_dict_re_matrix.head().T

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 26.9 µs


In [22]:
%%time

from IPython.display import display, Math

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import scipy.sparse as sp
from nltk.text import TextCollection

count_vect = CountVectorizer()
temp_matrix = count_vect.fit_transform(df_dict_re.values()) # temp_matrix эта промежуточная матрица, понадобится в следующем кейсе,
                                              # для вычисления tf-idf в данной матрице хранятся 
matrix_counts = temp_matrix.toarray()

words = [x[0] for x in sorted(count_vect.vocabulary_.items(), key=lambda x: x[1])] # список слов, 
                                                                                   # чтобы сделать красивую шапку
print(len(words))
# pd.DataFrame(matrix_counts, columns=words)  # при создании DataFrame передадим подготовленный список слов

CPU times: user 851 ms, sys: 1.96 s, total: 2.81 s
Wall time: 3.77 s


# Метод 2: через вектора слов 

Для новостной заметки можно составить ее словарь, а также посчитать частоты всех слов. В итоге мы получим представление текста в виде вектора. В этом векторе координаты будут называться по соответствующим словам, а смещение по данной координате будет показывать частота. <br>
При составлении словаря будем учитывать только значимые слова - существительные, прилагательные и глаголы. Помимо этого предусмотрим возможность учитывать часть речи слова, прибавляя ее у начальной форме.<br>
Для разделения текста на слова используем простейший алгоритм: слово - это последовательность букв русского алфавита среди которых может попадаться дефис. 

In [36]:
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.

morph = pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.

posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
meaningfullPoSes=['ADJF', 'NOUN', 'VERB']
def getArticleDictionary(text, needPos=None):
    words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    reswords=[]
    for w in words:
        wordform=morph.parse(w)[0]
        if wordform.tag.POS in meaningfullPoSes:
            if needPos!=None:
                reswords.append(wordform.normal_form+posConv[wordform.tag.POS])
            else:
                reswords.append(wordform.normal_form)
    return Counter(reswords)

# print(dict(getArticleDictionary(df_dict_re[1], True)))

In [104]:
%%time

df_dict_vector = {}
for line in df_dict.keys():
    df_dict_vector[line] = (dict(getArticleDictionary(df_dict[line], True)))
print('df_dict_vector ',len(df_dict_vector))

df_dict_vector  370179
CPU times: user 3h 54min 23s, sys: 33.2 s, total: 3h 54min 56s
Wall time: 3h 56min 8s


In [117]:
%%time

import yaml
# with open('df_dict.yml', 'w') as f:
#     yaml.dump(df_dict_vector, f, default_flow_style=False)
    
# with open('df_dict.yml', 'r') as f:
#     df_dict_vector = load(f)

CPU times: user 14 µs, sys: 23 µs, total: 37 µs
Wall time: 43.2 µs


In [37]:
import yaml

df_name_vector = {}
for line in df_name.keys():
    df_name_vector[line] = (dict(getArticleDictionary(df_name[line], True)))
print('df_name_vector ',len(df_name_vector))

with open('df_name.yml', 'w') as f:
    yaml.dump(df_name_vector, f, default_flow_style=False)

df_name_vector  370179
df_name to dict complete 



In [1]:
# %%time

# df_dict_vector_matrix = pd.DataFrame(list(df_dict_vector.values())).fillna(0)
# df_dict_vector_matrix.head(1).T

# Метод 4. TfidfVectorizer

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

counter=CountVectorizer()
# Просим посчитать частоты слов.
res=counter.fit_transform([df_dict[0]])

In [26]:
# При помощи ngram_range=(1,2) говорим, что хотим извлекать слова и пары слов.
# token_pattern показывает регулярное выражение, которому должны соответствовать слова.
counter12=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
# Проводим анализ, получаем список найденных n-грамм.
analyze = counter12.build_analyzer()
print(analyze(df_dict[0]))
# Считаем частоты, видим, что слова не приводились к начальной форме.
res=counter12.fit_transform([df_dict[0]])
print(counter12.vocabulary_.get('уверенный'))
# print(counter12.vocabulary_.get('мама'))

['обязанности', 'работа', 'с', 'клиентом', 'в', 'салоне', 'выезд', 'на', 'замер', 'создание', 'дизайн-проекта', 'расчеты', 'ведение', 'документации', 'заключение', 'договоров', 'требования', 'опыт', 'работы', 'желателен', 'уверенный', 'пользователь', 'пк', 'грамотная', 'речь', 'желание', 'учиться', 'и', 'развиваться', 'условия', 'трудоустройство', 'по', 'тк', 'обучение', 'стажировка', 'оклад', 'от', 'личных', 'продаж', 'удобный', 'график', 'обязанности работа', 'работа с', 'с клиентом', 'клиентом в', 'в салоне', 'салоне выезд', 'выезд на', 'на замер', 'замер создание', 'создание дизайн-проекта', 'дизайн-проекта расчеты', 'расчеты ведение', 'ведение документации', 'документации заключение', 'заключение договоров', 'договоров требования', 'требования опыт', 'опыт работы', 'работы желателен', 'желателен уверенный', 'уверенный пользователь', 'пользователь пк', 'пк грамотная', 'грамотная речь', 'речь желание', 'желание учиться', 'учиться и', 'и развиваться', 'развиваться условия', 'условия 

In [28]:
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.

morph = pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.

def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
    return words

c=[' '.join(getMeaningfullWords(df_dict[0]))]
c2=[' '.join([morph.parse(r)[0].normal_form for r in re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', df_dict[0])])]

lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = lemmaCounter.build_analyzer()
res1=analyze(c[0])
res2=lemmaCounter.fit_transform(c)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})
print("---")
res1=analyze(c2[0])
res2=lemmaCounter.fit_transform(c2)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})


{'работа': 2}
---
{'работа': 2}


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tfCounter=TfidfVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = tfCounter.build_analyzer()
res=tfCounter.fit_transform([df_dict[0]])
print(df_dict[0])

<p><strong>Обязанности:</strong></p> <ul> <li>Работа с клиентом в салоне,выезд на замер ,создание дизайн-проекта,расчеты,ведение документации,заключение договоров</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>Опыт работы желателен,уверенный пользователь ПК,грамотная речь,желание учиться и развиваться</li> </ul> <p> </p> <p><strong>Условия:</strong></p> <ul> <li>Трудоустройство по ТК, обучение+стажировка,оклад+% от личных продаж,удобный график</li> </ul>


In [17]:
id_article=6
res2=analyze(df_dict[1])
tfs=list(set(res[0, tfCounter.vocabulary_.get(k)] for k in res2 if k in tfCounter.vocabulary_.keys()))

In [31]:
# print({w:res[0, tfCounter.vocabulary_[w]] for w in res2 if w in tfCounter.vocabulary_.keys() and res[id_article][0, tfCounter.vocabulary_[w]] in tfs2})